In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tifffile import imread
from csbdeep.utils import download_and_extract_zip_file, plot_some
from csbdeep.data import RawData, create_patches
from tqdm import tqdm_notebook as tqdm
import os
import glob
from tifffile import imwrite

After downloading and unzipping the SEM data as described in the exercise sheet, let's have a look at it

In [ ]:
#For one image only
data = imread('data/SEM/train/train.tif')

fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20,16))
scantimes = ["0.2us", "0.5us", "1us", "1us", "2.1us", "5us", "5us, avg of 4"]
for idx, scantime in enumerate(scantimes):
    ax_idx = np.unravel_index(idx, (2,4))
    axs[ax_idx].imshow(data[idx], cmap="magma")
    axs[ax_idx].set_title(scantime)

<hr style="height:2px;">

# Generate training data for CARE

WUnlike in the original notebook, our data is not already organized into input (`low`) and target images (`GT`), we have to figure out how to do this ourselves.

First we wanna train a network for images of 1us scan time, so we will use those two images as our input images. For both of these images we can use every other image as our target - as long as the noise is different the only remaining structure is the signal, so mixing different scan times is totally fine.

Images are paired (as input and target) by having the same name in the `source_dirs` and `target_dir`. This means we'll have several copies of the same image with different names. These images aren't too big, so that's fine.

In [ ]:
if  len(glob.glob("data/SEM/train/low/*.tif"))>0:
    os.system("rm data/SEM/train/low/*.tif")
if len(glob.glob("data/SEM/train/GT/*.tif"))>0:
    os.system("rm data/SEM/train/GT/*.tif")
counter=0

for i in range(2,4):
    for j in range(data.shape[0]):
        if i == j:
            continue
        imwrite('data/SEM/train/low/'+str(counter)+'.tif',data[i,...])
        imwrite('data/SEM/train/GT/'+str(counter)+'.tif',data[j,...])
        counter+=1
        

Bonus: uncomment to instead use every image as an input, targeting with every other image

In [ ]:
# if  len(glob.glob("data/SEM/train/low/*.tif"))>0:
#     os.system("rm data/SEM/train/low/*.tif")
# if len(glob.glob("data/SEM/train/GT/*.tif"))>0:
#     os.system("rm data/SEM/train/GT/*.tif")
# counter=0

# for i in range(data.shape[0]):
#     for j in range(data.shape[0]):
#         if i == j:
#             continue
#         imwrite('data/SEM/train/low/'+str(counter)+'.tif',data[i,...])
#         imwrite('data/SEM/train/GT/'+str(counter)+'.tif',data[j,...])
#         counter+=1
    

In [ ]:
raw_data = RawData.from_folder (
    basepath    = 'data/SEM/train/',
    source_dirs = ['low'],
    target_dir  = 'GT',
    axes        = 'YX',
)



From corresponding stacks, we now generate some 2D patches.


In [ ]:
X, Y, XY_axes = create_patches (
    raw_data            = raw_data,
    patch_size          = (64,64),
    n_patches_per_image = 1024,
    save_file           = 'data/my_SEM_training_data.npz',
)

In [ ]:
assert X.shape == Y.shape
print("shape of X,Y =", X.shape)
print("axes  of X,Y =", XY_axes)

## Show

This shows the maximum projection of some of the generated patch pairs (odd rows: *source*, even rows: *target*)

In [ ]:
for i in range(2):
    plt.figure(figsize=(16,4))
    sl = slice(8*i, 8*(i+1)), 0
    plot_some(X[sl],Y[sl],title_list=[np.arange(sl[0].start,sl[0].stop)])
    plt.show()
None;